In [10]:
import pandas as pd
import numpy as np

In [12]:
df_match = pd.read_pickle("../Data/Match_Info/match_data.pickle")

In [13]:
df_match["participantIdentities"]

0        [{'summonerName': '쪼렙이다말로하자'}, {'summonerName'...
1        [{'summonerName': 'LPL Player'}, {'summonerNam...
2        [{'summonerName': '호잇이'}, {'summonerName': 'Ge...
3        [{'summonerName': '213321123'}, {'summonerName...
4        [{'summonerName': 'ggffggg'}, {'summonerName':...
                               ...                        
78115    [{'summonerName': 'SharK1r'}, {'summonerName':...
78116    [{'summonerName': '화가너무난사람'}, {'summonerName':...
78117    [{'summonerName': '만두은'}, {'summonerName': '18...
78118    [{'summonerName': '봇 주도권 있어'}, {'summonerName'...
78119    [{'summonerName': '우리처음만났던날'}, {'summonerName'...
Name: participantIdentities, Length: 78120, dtype: object

# Flattening the Dataframe
The runtimes will become very long, thats why the Dataframe will be split into different partitions. This way it is ensured, that the runtime wont crash and the memory keeps up with the work.

The variables in the funciton are:
- **df** is the dataframe used
- **column** is the column to be flattened
- **lower** lower limit
- **upper** upper limit
- **logical** is a logical flag, if set to true, it will take some other columns and append them to the df, if not, it will be used to just flatten the df.


In [66]:
def flattening_df(df,column:str, lower, upper, logical=True):
    flattened_df = pd.DataFrame()
    for i in range(lower,upper):
        if logical == True:
            flattened_df = (flattened_df.append(pd.json_normalize(df.loc[i,column]))
                                       .assign(gameDuration = df.loc[i,"gameDuration"]
                                            ,gameId=df.loc[i,"gameId"]
                                            ,gameVersion=df.loc[i,"gameVersion"]))
        else:
            flattened_df = flattened_df.append(pd.json_normalize(df.loc[i,column]))
    return flattened_df

# 2. Automatization of the Flattening process:
Taking stepsize of 100, that way the runtime stays short respectively to taking a lot of rows at the same time:

In [90]:
participants = pd.DataFrame()

In [91]:
stepsize = 100
for i in range(int(len(df_match)/stepsize)+1):
    lower = i*stepsize
    upper = (i+1)*stepsize
    dataframe = flattening_df(df_match,"participants", lower,upper)
    participants=participants.append(dataframe)
    if lower%1000==0:
        print(lower)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000


KeyError: 78120

In [92]:
participants.shape

(781000, 26)

In [93]:
names = pd.DataFrame()

In [94]:
stepsize = 100
for i in range(int(len(df_match)/stepsize)+1):
    lower = i*stepsize
    upper = (i+1)*stepsize
    names=names.append(flattening_df(df_match,"participantIdentities", lower,upper,logical=False))
    if lower%10000==0:
        print(lower)

0
10000
20000
30000
40000
50000
60000
70000


KeyError: 78120

In [97]:
names.head()

,summonerName
0,쪼렙이다말로하자
1,불질러
2,Youtube Thal
3,213321123
4,ggffggg


In [98]:
df_merge = participants.assign(summonerName = names['summonerName'])

In [109]:
df_merge.describe()

,participantId,teamId,championId,spell1Id,spell2Id,item0,item1,item2,item3,item4,...,deaths,assists,doubleKills,tripleKills,quadraKills,pentaKills,goldEarned,gameDuration,gameId,gameVersion
count,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,...,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,781000.000000,7.810000e+05,781000.000000
mean,5.500000,150.000000,176.067536,8.746869,6.300593,2457.471488,2486.960835,2331.996772,2285.744734,2101.759763,...,4.367097,6.810312,0.409589,0.054018,0.007090,0.001014,9196.573973,1454.293214,4.165494e+09,10.311344
std,2.872283,50.000032,184.683520,4.399838,3.874694,1030.901711,1037.014765,1153.502038,1152.934730,1229.503359,...,2.743135,5.185401,0.769363,0.249042,0.086456,0.032029,3573.316417,408.577986,9.679835e+07,0.395348
min,1.000000,100.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,664.000000,192.000000,3.384871e+09,8.200000
25%,3.000000,100.000000,58.000000,4.000000,4.000000,1413.000000,2031.000000,1082.000000,1055.000000,1038.000000,...,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,6646.000000,1174.000000,4.136710e+09,10.300000
50%,5.500000,150.000000,104.000000,11.000000,4.000000,3031.000000,3046.000000,3026.000000,3027.000000,3020.000000,...,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,8903.000000,1446.000000,4.193794e+09,10.500000
75%,8.000000,200.000000,236.000000,12.000000,7.000000,3152.000000,3151.000000,3133.000000,3111.000000,3109.000000,...,6.000000,10.000000,1.000000,0.000000,0.000000,0.000000,11526.000000,1735.000000,4.223771e+09,10.500000
max,10.000000,200.000000,875.000000,21.000000,21.000000,3916.000000,3916.000000,3916.000000,3916.000000,3916.000000,...,23.000000,47.000000,10.000000,5.000000,3.000000,2.000000,31957.000000,2969.000000,4.256800e+09,10.600000


In [ ]:
df_merge.to_pickle("../Data/Match_Info/match_data.pickle")